In [1]:
import datetime as dt
import pandas as pd
import numpy as np
import scipy.stats as st
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
UNWANTED_COLUMNS = ["amenities", "address", "pets_allowed", "body", "has_photo","source","price_display", "title", "category"]
POSTPROCESING_UNWANTED_COLUMNS = ["price_type", "currency", "time"]
RAW_DATA_PATH = "../raw_data/data.csv"
CLEAN_DATA_PATH = "../clean_data/clean_data.csv"
GEO_COLUMNS = ["latitude", "longitude"]
GEO_DECIMAL_CORRECTION = 0.1
VALID_PRICE_TYPES = ['Monthly',]
VALID_CURRENCIES = ['USD',]

In [3]:
def clean_lon(string):
    dot_count = string.count(".")
    if dot_count > 2:
        raise ValueError(f"Invalid number of decimals {dot_count} in {string}")
    elif dot_count == 2:
        clean_string = string.replace(".", "", 1)
    else:
        clean_string = string
    return float(clean_string) 

In [4]:
# Read in the data.
df = pd.read_csv(RAW_DATA_PATH, low_memory=False)

In [5]:
df = df.drop(columns=UNWANTED_COLUMNS).dropna()
df = df[df['price_type'].isin(VALID_PRICE_TYPES)]
df = df[df['currency'].isin(VALID_CURRENCIES)]
df['date'] = df['time'].map(dt.date.fromtimestamp)
df.drop(columns=POSTPROCESING_UNWANTED_COLUMNS, inplace=True) 
df.longitude = df.longitude.map(clean_lon)
#Make the latitude and longitude decimal right again
for col in GEO_COLUMNS:
    df[col] = df[col] * GEO_DECIMAL_CORRECTION

print(df.shape)
df.head(3)

(99004, 11)


,id,bathrooms,bedrooms,fee,price,square_feet,cityname,state,latitude,longitude,date
0,5668640009,1.0,1.0,No,2195.0,542,Redondo Beach,CA,33.8520,-118.3759,2019-12-26
1,5668639818,1.5,3.0,No,1250.0,1500,Newport News,VA,37.0867,-76.4941,2019-12-26
2,5668639686,2.0,3.0,No,1395.0,1650,Raleigh,NC,35.8230,-78.6438,2019-12-26


In [6]:
df["price_per_square_ft"] = df.price / df.square_feet

In [7]:
df.shape
df.head()

,id,bathrooms,bedrooms,fee,price,square_feet,cityname,state,latitude,longitude,date,price_per_square_ft
0,5668640009,1.0,1.0,No,2195.0,542,Redondo Beach,CA,33.8520,-118.3759,2019-12-26,4.049815
1,5668639818,1.5,3.0,No,1250.0,1500,Newport News,VA,37.0867,-76.4941,2019-12-26,0.833333
2,5668639686,2.0,3.0,No,1395.0,1650,Raleigh,NC,35.8230,-78.6438,2019-12-26,0.845455
3,5668639659,1.0,2.0,No,1600.0,820,Vacaville,CA,38.3622,-121.9712,2019-12-26,1.951220
4,5668639374,1.0,1.0,No,975.0,624,Albuquerque,NM,35.1038,-106.6110,2019-12-26,1.562500


In [8]:
df.to_csv(CLEAN_DATA_PATH)